In [34]:
!pip3 install pyocto pyarrow

  Could not find a version that satisfies the requirement pyocto (from versions: )
No matching distribution found for pyocto


In [60]:
import pyocto
import pandas as pd
import datetime
import matplotlib.pyplot as plt

## Input Data

In [94]:
picks = pd.read_csv("~/notebooks/data/phase_association/pick")
stations = pd.read_csv("~/notebooks/data/phase_association/stations")

In [95]:
picks

,Unnamed: 0,station,phase,time
0,0,FS09B,P,2012-10-01T00:04:09.141700Z
1,1,FS09B,P,2012-10-01T00:36:41.601700Z
2,2,FS09B,P,2012-10-01T00:55:43.061700Z
3,3,FS09B,P,2012-10-01T01:16:07.681700Z
4,4,FS09B,P,2012-10-01T01:41:37.561700Z
...,...,...,...,...
2444,2444,M14B,S,2012-10-01T19:43:53.779925Z
2445,2445,M14B,S,2012-10-01T21:19:41.539925Z
2446,2446,M14B,S,2012-10-01T21:38:06.599925Z
2447,2447,M14B,S,2012-10-01T23:08:32.799925Z


  ## Velocoty Models

In [96]:
velocity_model = pyocto.VelocityModel0D(
    p_velocity=7.0,
    s_velocity=4.0,
    tolerance=2.0,
)

## The OctoAssociator class

In [97]:
associator = pyocto.OctoAssociator.from_area(
    lat=(42, 50),
    lon=(-126, -123),
    zlim=(0, 200),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=10,
    n_p_and_s_picks=4,
)

In [98]:
associator.crs

<Projected CRS: +proj=tmerc +lat_0=46.0 +lon_0=-124.5 +units=km +t ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (kilometre)
- N[north]: Northing (kilometre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [99]:
associator.transform_stations(stations)
stations

,Unnamed: 0,id,longitude,latitude,elevation,x,y,z
0,0,FS09B,-124.808502,40.438702,-2161.0,-26.174832,-617.798222,2.161
1,1,FS13B,-124.806503,40.493099,-2332.0,-25.984256,-611.758360,2.332
2,2,G10B,-125.553299,40.677898,-2936.0,-89.050281,-590.748764,2.936
3,3,G19B,-125.773598,41.307400,-3071.0,-106.656163,-520.591199,3.071
4,4,G25B,-124.586098,41.919899,-430.0,-7.142224,-453.342481,0.430
5,5,G27B,-126.016701,41.916599,-3480.0,-125.825392,-452.599904,3.480
6,6,J09B,-124.726997,43.151001,-252.0,-18.464168,-316.565691,0.252
7,7,M12B,-124.946098,42.183998,-1045.0,-36.852931,-423.915116,1.045
8,8,M14B,-124.589699,40.985001,-638.0,-7.548553,-557.174455,0.638


## Convert the timestamps to floats

In [100]:
picks["time"] = picks["time"].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%fZ'))

In [101]:
picks["time"] = picks["time"].apply(lambda x: x.timestamp())
picks

,Unnamed: 0,station,phase,time
0,0,FS09B,P,1.349050e+09
1,1,FS09B,P,1.349052e+09
2,2,FS09B,P,1.349053e+09
3,3,FS09B,P,1.349054e+09
4,4,FS09B,P,1.349056e+09
...,...,...,...,...
2444,2444,M14B,S,1.349121e+09
2445,2445,M14B,S,1.349126e+09
2446,2446,M14B,S,1.349127e+09
2447,2447,M14B,S,1.349133e+09


In [102]:
type(1.349050e+09)

float

## Delete unnecessary columns

In [103]:
picks.drop(columns="Unnamed: 0", inplace=True)
picks

,station,phase,time
0,FS09B,P,1.349050e+09
1,FS09B,P,1.349052e+09
2,FS09B,P,1.349053e+09
3,FS09B,P,1.349054e+09
4,FS09B,P,1.349056e+09
...,...,...,...
2444,M14B,S,1.349121e+09
2445,M14B,S,1.349126e+09
2446,M14B,S,1.349127e+09
2447,M14B,S,1.349133e+09


In [104]:
stations.drop(columns="Unnamed: 0", inplace=True)
stations

,id,longitude,latitude,elevation,x,y,z
0,FS09B,-124.808502,40.438702,-2161.0,-26.174832,-617.798222,2.161
1,FS13B,-124.806503,40.493099,-2332.0,-25.984256,-611.758360,2.332
2,G10B,-125.553299,40.677898,-2936.0,-89.050281,-590.748764,2.936
3,G19B,-125.773598,41.307400,-3071.0,-106.656163,-520.591199,3.071
4,G25B,-124.586098,41.919899,-430.0,-7.142224,-453.342481,0.430
5,G27B,-126.016701,41.916599,-3480.0,-125.825392,-452.599904,3.480
6,J09B,-124.726997,43.151001,-252.0,-18.464168,-316.565691,0.252
7,M12B,-124.946098,42.183998,-1045.0,-36.852931,-423.915116,1.045
8,M14B,-124.589699,40.985001,-638.0,-7.548553,-557.174455,0.638


## Apply  the associator

In [105]:
events, assignments = associator.associate(picks, stations)

In [106]:
events

""


In [107]:
assignments

,event_idx,pick_idx,residual,station,phase,time
